In [ ]:
import json
import re

from pyspark.sql import SparkSession

# Create or retrieve a SparkSession
spark = SparkSession.builder.appName("ChiSquaredRDD").getOrCreate()
sc = spark.sparkContext

# Load the review file as an RDD
# review_path = "hdfs:///user/dic23_shared/amazon-reviews/full/reviewscombined.json"
review_path = "hdfs:///user/dic23_shared/amazon-reviews/full/reviews_devset.json"
# review_path = "hdfs:///user/e11809642/reviews/reduced_devset.json"
input_rdd = sc.textFile(review_path)

# Load the stopword file as a set
stopwords_path = "hdfs:///user/e11809642/stopwords.txt"
stopwords = set(sc.textFile(stopwords_path).collect())

In [ ]:
# Parse the JSON strings and extract the category and review text
category_review_rdd = input_rdd \
    .map(lambda json_str: json.loads(json_str)) \
    .map(lambda json_obj: (json_obj['category'], json_obj['reviewText']))

In [ ]:
# Compute the total number of documents
review_count = category_review_rdd.count()

# Compute the number of documents in each category
category_counts_rdd = category_review_rdd \
    .map(lambda pair: (pair[0], 1)) \
    .reduceByKey(lambda x, y: x + y)

In [ ]:
# Compute the chi-squared value for a given term and category
def calculate_chi_square(term, category):
    a = term_category_occurrences_rdd[(term, category)]
    b = term_occurrences_rdd[term] - a
    c = category_counts_rdd[category] - a
    d = review_count - a - b - c
    return review_count * (a * d - b * c) ** 2 / ((a + b) * (a + c) * (b + d) * (c + d))

In [ ]:
# Tokenize the reviewText of each review based on a regex
pattern = re.compile(r"[^a-zA-Z<>^|]+")

# Compute the number of occurrences of each term in each category across all reviews
term_category_occurrences_rdd = category_review_rdd \
    .map(lambda pair: ((pair[0], pair[1]), set(pattern.split(pair[1])))) \
    .flatMap(lambda pair: (((term.lower(), pair[0][0]), 1) for term in pair[1] if
                           term.lower() not in stopwords and len(term) >= 2)) \
    .reduceByKey(lambda x, y: x + y)

# print(term_category_occurrences_rdd.take(10))

# Compute the number of occurrences of each term across all reviews
term_occurrences_rdd = term_category_occurrences_rdd \
    .map(lambda pair: (pair[0][0], pair[1])) \
    .reduceByKey(lambda x, y: x + y)

# print(term_occurrences_rdd.take(10))

# Compute the chi-squared value for each unique term and category pair
# (term, category) -> chi-square
term_category_chi_squared_rdd = term_category_occurrences_rdd \
    .map(lambda pair: (pair[0][0], (pair[0][1], calculate_chi_square(pair[0][0], pair[0][1])))) \
    .groupByKey()

# print(term_category_chi_squared_rdd.take(10))

# Select the top 75 tokens with the highest chi-square value for each category
# (category) -> [(token, chi-square)]
chi_square_rdd = term_category_chi_squared_rdd \
    .map(lambda pair: (pair[0], sorted(pair[1], key=lambda x: x[1], reverse=True)[:75]))

# Select all unique tokens from the top 75 tokens with the highest chi-square value for each category
tokens = chi_square_rdd \
    .flatMap(lambda pair: (token for token, chi_square in pair[1])) \
    .distinct() \
    .collect()

# Sort the tokens in alphabetical order
tokens.sort()

# Sort the categories in alphabetical order
chi_square_rdd = chi_square_rdd.sortByKey()

# Save the top 75 tokens with the highest chi-square value for each category to a file in the local file system
# in the format: "<category> term1:chi_squared1 term2:chi_squared2 ... term75:chi_squared75" for each line and append the list of tokens to the end of the file
with open("chi_squared.txt", "a") as file:
    for pair in chi_square_rdd.collect():
        file.write("<%s>" % pair[0] + " ")
        for token, chi_square in pair[1]:
            file.write("%s:%f" % (token, chi_square) + " ")
        file.write("\n")
    file.write(" ".join(tokens) + "\n")

In [ ]:
sc.stop()
